In [ ]:
import sagemaker

region = sagemaker.Session().boto_region_name
print("AWS Region: {}".format(region))

role = sagemaker.get_execution_role()
print("RoleArn: {}".format(role))

In [ ]:
import boto3, os

In [ ]:
!pip install python-dotenv
%reload_ext dotenv
%dotenv

In [ ]:
# Access and Credential details
bucket_name = os.environ['BUCKET_NAME'] # <--- Environment variable for Bucket Name
ecs_access_key_id=os.environ['ECS_ACCESS_KEY_ID']  # <--- Environment variable for ECS Access Key
ecs_secret_access_key=os.environ['ECS_SECRET_ACCESS_KEY'] # <--- Environment variable for Secret Access Key
endpoint_url=os.environ['ENDPOINT_URL'] # <--- Environment variable for Endpoint URL

In [ ]:
s3=boto3.resource(service_name='s3',aws_access_key_id=ecs_access_key_id,aws_secret_access_key=ecs_secret_access_key,endpoint_url=endpoint_url) 
object_to_train=s3.Bucket(bucket_name).Object(os.path.join('data/train.csv')).download_file('train.csv')
Object_to_validate=s3.Bucket(bucket_name).Object(os.path.join('data/validation.csv')).download_file('validation.csv')

In [ ]:
bucket = sagemaker.Session().default_bucket()
prefix = "demo-sagemaker-xgboost-adult-income-prediction"

boto3.Session().resource('s3').Bucket(bucket).Object(
    os.path.join(prefix, 'data/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(
    os.path.join(prefix, 'data/validation.csv')).upload_file('validation.csv')

In [ ]:
from sagemaker.debugger import Rule, ProfilerRule, rule_configs
from sagemaker.session import TrainingInput

s3_output_location='s3://{}/{}/{}'.format(bucket, prefix, 'xgboost_model')

container=sagemaker.image_uris.retrieve("xgboost", region, "1.2-1")
print(container)

xgb_model=sagemaker.estimator.Estimator(
    image_uri=container,
    role=role,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    volume_size=5,
    output_path=s3_output_location,
    sagemaker_session=sagemaker.Session(),
    rules=[
        Rule.sagemaker(rule_configs.create_xgboost_report()),
        ProfilerRule.sagemaker(rule_configs.ProfilerReport())
    ]
)

In [ ]:
xgb_model.set_hyperparameters(
    max_depth = 5,
    eta = 0.2,
    gamma = 4,
    min_child_weight = 6,
    subsample = 0.7,
    objective = "binary:logistic",
    num_round = 1000
)

In [ ]:
from sagemaker.session import TrainingInput

train_input = TrainingInput(
    "s3://{}/{}/{}".format(bucket, prefix, "data/train.csv"), content_type="csv"
)
validation_input = TrainingInput(
    "s3://{}/{}/{}".format(bucket, prefix, "data/validation.csv"), content_type="csv"
)

In [ ]:
xgb_model.fit({"train": train_input, "validation": validation_input}, wait=True)

In [ ]:
rule_output_path = xgb_model.output_path + "/" + xgb_model.latest_training_job.job_name + "/rule-output"
! aws s3 ls {rule_output_path} --recursive

In [ ]:
! aws s3 cp {rule_output_path} ./ --recursive

In [ ]:
from IPython.display import FileLink, FileLinks
display("Click link below to view the XGBoost Training report", FileLink("CreateXgboostReport/xgboost_report.html"))

In [ ]:
profiler_report_name = [rule["RuleConfigurationName"] 
                        for rule in xgb_model.latest_training_job.rule_job_summary() 
                        if "Profiler" in rule["RuleConfigurationName"]][0]
profiler_report_name
display("Click link below to view the profiler report", FileLink(profiler_report_name+"/profiler-output/profiler-report.html"))

In [ ]:
xgb_model.model_data

In [ ]:
# Access and Credential details
bucket_name = os.environ['BUCKET_NAME'] # <--- Environment variable for Bucket Name
ecs_access_key_id=os.environ['ECS_ACCESS_KEY_ID']  # <--- Environment variable for ECS Access Key
ecs_secret_access_key=os.environ['ECS_SECRET_ACCESS_KEY'] # <--- Environment variable for Secret Access Key
endpoint_url=os.environ['ENDPOINT_URL'] # <--- Environment variable for Endpoint URL

upload_report=s3.Bucket(bucket_name).Object(os.path.join('profiler_output/profiler-report.html')).upload_file('./ProfilerReport/profiler-output/profiler-report.html')
upload_profiler=s3.Bucket(bucket_name).Object(os.path.join('profiler_output/profiler-report.ipynb')).upload_file('./ProfilerReport/profiler-output/profiler-report.ipynb')